In [22]:
!pip install pandas_datareader

In [45]:
import requests
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
import plotly.express as px

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/analyticsariel/market-research-data/main/cyber_security_jobs_indeed_fl_202406.csv')
print('Number of columns:', len(df.columns))
print('Number of rows:', len(df))
df.head(10)

Number of columns: 8
Number of rows: 180


,Position,Title,Company,Location,Salary,Short Description,Posted At,Detail link
0,1,Cybersecurity Engineer I,South State Bank,"Winter Haven, FL",NaN,Recommend and implement additional security so...,Posted\nPosted 5 days ago,https://www.indeed.com/rc/clk?jk=adb095e98503d...
1,2,Cyber Security Engineer - Remote (WFH),COGNITIVE MEDICAL SYSTEMS INC,"Remote in Tampa, FL 33619","$115,000 - $140,000 a year",Determine security requirements by evaluating ...,Posted\nPosted 9 days ago,https://www.indeed.com/rc/clk?jk=d747d53eb8824...
2,3,Information Security Engineer I,Ryder System,"Miami, FL 33178",NaN,"Practical experience with database security, c...",Posted\nPosted 3 days ago,https://www.indeed.com/rc/clk?jk=4371c17cf0d79...
3,4,Data Security Engineer,City National Bank of Florida,"Miami, FL 33131 \n(Downtown area)",NaN,2-4 years of related security systems administ...,Posted\nPosted 2 days ago,https://www.indeed.com/rc/clk?jk=f8356c8cfc42b...
4,5,IT Security Engineer,Mount Sinai Medical Center - Florida,"Miami Beach, FL 33140 \n(Nautilus area)",NaN,"Evaluate, recommend, implement network and sec...",Posted\nPosted 3 days ago,https://www.indeed.com/rc/clk?jk=ae03e412bee13...
5,6,Cyber Security Engineer,Cinch Home Services,"Hybrid work in Boca Raton, FL 33431",NaN,Minimum 5+ years of experience in cyber securi...,Employer\nActive 3 days ago,https://www.indeed.com/rc/clk?jk=1bb646ca9199e...
6,7,Cybersecurity Engineer,ThreatLocker,"Orlando, FL",NaN,Respond to security incidents promptly.\nDevel...,Posted\nPosted 1 day ago,https://www.indeed.com/rc/clk?jk=dd6bfe3a1e761...
7,8,Application Security Engineer,"Applied Research Associates, Inc","Niceville, FL 32578",NaN,"Familiar with common security libraries, RMF s...",Posted\nPosted 30+ days ago,https://www.indeed.com/rc/clk?jk=4855a7a967309...
8,9,Security Engineer,AnyDesk Software GmbH,"Tampa, FL",NaN,Proven experience in a security engineering ro...,Posted\nPosted 10 days ago,https://www.indeed.com/rc/clk?jk=08cacfef3c2c8...
9,10,Network Security Engineer,Health Care District of Palm Beach County,"West Palm Beach, FL 33401",NaN,Minimum 5 years experience in the network secu...,Posted\nPosted 11 days ago,https://www.indeed.com/rc/clk?jk=001858568849b...


In [25]:
df_feat = df.copy()
df_feat['date_retrieved'] = '2024-06-15' # change this to the date you pull the data
# get days posted
df_feat['days_posted_regex_match'] = df_feat.apply(lambda x: re.search(r"Posted (\d)", x['Posted At']), axis=1)
df_feat['date_posted'] = df_feat.apply(lambda x: datetime.strptime(x['date_retrieved'], '%Y-%m-%d') - timedelta(days=int(x['days_posted_regex_match'].group(1))) if x['days_posted_regex_match'] else None, axis=1)
# get city
df_feat['city_regex_match'] = df_feat.apply(lambda x: re.search(r"(\b\w+\s*),\s*(?:[A-Z]{2,}|[^,]+)" , x['Location']), axis=1)
df_feat['city_name'] = df_feat.apply(lambda x: x['city_regex_match'].group(1) if x['city_regex_match'] else None, axis=1)
# get work from home flag
df_feat['wfh_flag'] = df_feat.apply(lambda x: True if 'wfh' in x['Title'].lower() else False, axis=1)
df_feat = df_feat.drop(columns=['days_posted_regex_match', 'city_regex_match', 'Posted At'])
df_feat.head(10)

,Position,Title,Company,Location,Salary,Short Description,Detail link,date_retrieved,date_posted,city_name,wfh_flag
0,1,Cybersecurity Engineer I,South State Bank,"Winter Haven, FL",NaN,Recommend and implement additional security so...,https://www.indeed.com/rc/clk?jk=adb095e98503d...,2024-06-15,2024-06-10,Haven,False
1,2,Cyber Security Engineer - Remote (WFH),COGNITIVE MEDICAL SYSTEMS INC,"Remote in Tampa, FL 33619","$115,000 - $140,000 a year",Determine security requirements by evaluating ...,https://www.indeed.com/rc/clk?jk=d747d53eb8824...,2024-06-15,2024-06-06,Tampa,True
2,3,Information Security Engineer I,Ryder System,"Miami, FL 33178",NaN,"Practical experience with database security, c...",https://www.indeed.com/rc/clk?jk=4371c17cf0d79...,2024-06-15,2024-06-12,Miami,False
3,4,Data Security Engineer,City National Bank of Florida,"Miami, FL 33131 \n(Downtown area)",NaN,2-4 years of related security systems administ...,https://www.indeed.com/rc/clk?jk=f8356c8cfc42b...,2024-06-15,2024-06-13,Miami,False
4,5,IT Security Engineer,Mount Sinai Medical Center - Florida,"Miami Beach, FL 33140 \n(Nautilus area)",NaN,"Evaluate, recommend, implement network and sec...",https://www.indeed.com/rc/clk?jk=ae03e412bee13...,2024-06-15,2024-06-12,Beach,False
5,6,Cyber Security Engineer,Cinch Home Services,"Hybrid work in Boca Raton, FL 33431",NaN,Minimum 5+ years of experience in cyber securi...,https://www.indeed.com/rc/clk?jk=1bb646ca9199e...,2024-06-15,NaT,Raton,False
6,7,Cybersecurity Engineer,ThreatLocker,"Orlando, FL",NaN,Respond to security incidents promptly.\nDevel...,https://www.indeed.com/rc/clk?jk=dd6bfe3a1e761...,2024-06-15,2024-06-14,Orlando,False
7,8,Application Security Engineer,"Applied Research Associates, Inc","Niceville, FL 32578",NaN,"Familiar with common security libraries, RMF s...",https://www.indeed.com/rc/clk?jk=4855a7a967309...,2024-06-15,2024-06-12,Niceville,False
8,9,Security Engineer,AnyDesk Software GmbH,"Tampa, FL",NaN,Proven experience in a security engineering ro...,https://www.indeed.com/rc/clk?jk=08cacfef3c2c8...,2024-06-15,2024-06-14,Tampa,False
9,10,Network Security Engineer,Health Care District of Palm Beach County,"West Palm Beach, FL 33401",NaN,Minimum 5 years experience in the network secu...,https://www.indeed.com/rc/clk?jk=001858568849b...,2024-06-15,2024-06-14,Beach,False


In [26]:
# top employers with job postings
df_feat.groupby(['Company', 'city_name'])['Title'].count().sort_values(ascending=False).reset_index().head(10)

,Company,city_name,Title
0,BNY Mellon,Mary,5
1,Deloitte,Raton,5
2,"MUFG Bank, Ltd.",Tampa,5
3,Argo Cyber Systems,Pensacola,4
4,ServiceNow,Orlando,4
5,DGR Systems LLC,Tampa,3
6,BankUnited,Lakes,3
7,"ICR, Inc.",Orlando,3
8,Deepwatch,Tampa,3
9,Axient LLC,AFB,3


In [27]:
df_feat.loc[df_feat['Company'].str.contains('L3Harris')]

,Position,Title,Company,Location,Salary,Short Description,Detail link,date_retrieved,date_posted,city_name,wfh_flag
70,71,"Lead, Information Security Systems Engineer (I...",L3Harris Technologies,"Melbourne, FL",NaN,Experience developing network security archite...,https://www.indeed.com/rc/clk?jk=6013018e623e9...,2024-06-15,2024-06-14,Melbourne,False
73,74,"Electronic Warfare, Systems Engineering Specia...",L3Harris Technologies,"Palm Bay, FL",NaN,"For 60 years, customers have turned to L3Harri...",https://www.indeed.com/rc/clk?jk=a929f618fa093...,2024-06-15,2024-06-14,Bay,False
163,164,"Scientist, Info Sec Systems Engineer - Palm Ba...",L3Harris Technologies,"Palm Bay, FL",NaN,Assist program security in the development of ...,https://www.indeed.com/rc/clk?jk=7f87cd965ffff...,2024-06-15,2024-06-12,Bay,False
170,171,"Lead, Systems Engineer - Cyber Effects (Active...",L3Harris Technologies,"Palm Bay, FL",NaN,"As you develop operational CONOPS, you will wo...",https://www.indeed.com/rc/clk?jk=4552315d3c0fe...,2024-06-15,2024-06-12,Bay,False


In [28]:
# work from home roles
df_feat.groupby(['wfh_flag'])['Title'].count().sort_values(ascending=False).reset_index()

,wfh_flag,Title
0,False,179
1,True,1


In [29]:
# number of roles per city
df_grp_jobs_by_city = df_feat.groupby(['city_name'])['Title'].count().sort_values(ascending=False).reset_index().rename(columns={'Title': 'job_count'})
df_grp_jobs_by_city.head(10)

,city_name,job_count
0,Tampa,34
1,Orlando,22
2,Miami,17
3,Pensacola,14
4,Jacksonville,11
5,Melbourne,8
6,Tallahassee,7
7,Beach,7
8,Raton,6
9,Bay,6


In [30]:
# create bar chart to view spread of roles by city
fig = px.bar(df_grp_jobs_by_city, x='city_name', y='job_count', title='Number of Cyber Security roles by city (FL)')
fig.show()

In [31]:
print(df.columns)

Index(['Position', 'Title', 'Company', 'Location', 'Salary',
       'Short Description', 'Posted At', 'Detail link'],
      dtype='object')


In [36]:
population_data = {
    'Location': ['California', 'Texas', 'New York', 'Florida', 'Illinois'],
    'total_population': [39538223, 29145505, 20201249, 21538187, 12812508]
}

population_df = pd.DataFrame(population_data)

In [39]:
# Add flags based on job title
df['Title_lower'] = df['Title'].str.lower()

# 1 if job is cyber-related
df['is_cyber'] = df['Title_lower'].apply(lambda x: 1 if 'cyber' in x or 'security' in x else 0)

# Assuming all jobs are tech jobs
df['is_tech'] = 1

In [40]:
population_df

,Location,total_population
0,California,39538223
1,Texas,29145505
2,New York,20201249
3,Florida,21538187
4,Illinois,12812508


In [41]:
# Aggregate job counts by location
job_counts = df.groupby('Location').agg({
    'is_cyber': 'sum',
    'is_tech': 'sum'
}).reset_index()

# Rename for clarity
job_counts.rename(columns={
    'is_cyber': 'cyber_jobs',
    'is_tech': 'tech_jobs'
}, inplace=True)

In [42]:
#merge population data
final_df = job_counts.merge(population_df, on='Location', how='left')

In [43]:
final_df

,Location,cyber_jobs,tech_jobs,total_population
0,"Altamonte Springs, FL 32701",1,1,NaN
1,"Boca Raton, FL",2,2,NaN
2,"Cape Canaveral, FL",0,1,NaN
3,"Daytona Beach, FL 32114",0,1,NaN
4,"Destin, FL",1,1,NaN
...,...,...,...,...
93,"Tampa, FL 33613",0,1,NaN
94,"Tampa, FL 33621 \n(MacDill AFB Area area)",0,1,NaN
95,"West Palm Beach, FL 33401",1,1,NaN
96,"Winter Haven, FL",1,1,NaN


In [46]:
final_df['ratio_cyber_to_tech'] = final_df['cyber_jobs'] / final_df['tech_jobs'].replace(0, np.nan)
final_df['ratio_cyber_to_population'] = final_df['cyber_jobs'] / final_df['total_population'].replace(0, np.nan)

In [47]:
final_df

,Location,cyber_jobs,tech_jobs,total_population,ratio_cyber_to_tech,ratio_cyber_to_population
0,"Altamonte Springs, FL 32701",1,1,NaN,1.0,NaN
1,"Boca Raton, FL",2,2,NaN,1.0,NaN
2,"Cape Canaveral, FL",0,1,NaN,0.0,NaN
3,"Daytona Beach, FL 32114",0,1,NaN,0.0,NaN
4,"Destin, FL",1,1,NaN,1.0,NaN
...,...,...,...,...,...,...
93,"Tampa, FL 33613",0,1,NaN,0.0,NaN
94,"Tampa, FL 33621 \n(MacDill AFB Area area)",0,1,NaN,0.0,NaN
95,"West Palm Beach, FL 33401",1,1,NaN,1.0,NaN
96,"Winter Haven, FL",1,1,NaN,1.0,NaN


In [49]:
# Which locations have the highest concentration of cyber jobs?
top_cyber_locations = final_df.sort_values(by='cyber_jobs', ascending=False).head(10)
top_cyber_locations

,Location,cyber_jobs,tech_jobs,total_population,ratio_cyber_to_tech,ratio_cyber_to_population
52,"Orlando, FL",8,9,NaN,0.888889,NaN
60,"Pensacola, FL",6,8,NaN,0.750000,NaN
85,"Tampa, FL",6,8,NaN,0.750000,NaN
45,"Miami, FL",5,6,NaN,0.833333,NaN
54,"Orlando, FL 32826",5,6,NaN,0.833333,NaN
76,"Remote in Tampa, FL",5,6,NaN,0.833333,NaN
23,"Hybrid work in Tampa, FL 33607",4,4,NaN,1.000000,NaN
32,"Lake Mary, FL",4,5,NaN,0.800000,NaN
21,"Hybrid work in Tampa, FL",3,3,NaN,1.000000,NaN
25,"Jacksonville, FL",3,3,NaN,1.000000,NaN


In [53]:
import re

# Clean the location names
def standardize_location(location):
    match = re.search(r"([A-Za-z\s]+),\s*[A-Z]{2}", location)
    return match.group(0) if match else location

final_df['clean_location'] = final_df['Location'].apply(standardize_location)

In [54]:
population_df['clean_location'] = population_df['Location']

final_df = final_df.drop(columns='total_population')  # Drop existing NaNs
final_df = final_df.merge(population_df[['clean_location', 'total_population']],
                          on='clean_location', how='left')

In [55]:
# Fill all NaNs with the median population (if known)
final_df['total_population'].fillna(final_df['total_population'].median(), inplace=True)

/tmp/ipython-input-55-202844497.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [57]:
final_df['ratio_cyber_to_population'] = final_df['cyber_jobs'] / final_df['total_population'].replace(0, np.nan)
final_df

,Location,cyber_jobs,tech_jobs,ratio_cyber_to_tech,ratio_cyber_to_population,clean_location,total_population
0,"Altamonte Springs, FL 32701",1,1,1.0,4.642916e-08,"Altamonte Springs, FL",21538187.0
1,"Boca Raton, FL",2,2,1.0,9.285833e-08,"Boca Raton, FL",21538187.0
2,"Cape Canaveral, FL",0,1,0.0,0.000000e+00,"Cape Canaveral, FL",21538187.0
3,"Daytona Beach, FL 32114",0,1,0.0,0.000000e+00,"Daytona Beach, FL",21538187.0
4,"Destin, FL",1,1,1.0,4.642916e-08,"Destin, FL",21538187.0
...,...,...,...,...,...,...,...
93,"Tampa, FL 33613",0,1,0.0,0.000000e+00,"Tampa, FL",21538187.0
94,"Tampa, FL 33621 \n(MacDill AFB Area area)",0,1,0.0,0.000000e+00,"Tampa, FL",21538187.0
95,"West Palm Beach, FL 33401",1,1,1.0,4.642916e-08,"West Palm Beach, FL",21538187.0
96,"Winter Haven, FL",1,1,1.0,4.642916e-08,"Winter Haven, FL",21538187.0


In [59]:
df.isnull().sum()

,0
Position,0
Title,0
Company,0
Location,0
Salary,127
Short Description,0
Posted At,0
Detail link,0
Title_lower,0
is_cyber,0


In [60]:
median_salary = df['Salary_clean'].median()
df['Salary_clean'].fillna(median_salary, inplace=True)

/tmp/ipython-input-60-1137178060.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [61]:
mean_salary = df['Salary_clean'].mean()
df['Salary_clean'].fillna(mean_salary, inplace=True)

/tmp/ipython-input-61-667256605.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [62]:
df['Salary_clean'] = df.groupby('is_cyber')['Salary_clean'].transform(
    lambda x: x.fillna(x.median())
)

In [63]:
df = df.dropna(subset=['Salary_clean'])

In [64]:
df.isnull().sum()

,0
Position,0
Title,0
Company,0
Location,0
Salary,0
Short Description,0
Posted At,0
Detail link,0
Title_lower,0
is_cyber,0
